# Extract SourcesTables Results in  OGA

 work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/01/19
- update : 2023/01/20


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_01/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g754a7f0350+935911ccad 	current w_2023_01 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

In [8]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [9]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
file_out="sourceTable_202301-17-18.csv"

## Give the collection

In [11]:
#my_collection = "u/dagoret/BPS_test"
my_collection = "u/dagoret/BPS_photimaging_202301"

In [12]:
#butler = butlerUtils.makeDefaultLatissButler(extraCollections=[my_collection])

## Config

In [13]:
# path index for each month
#DATE="20211103"
#filterdispersername = "empty~holo4_003"
#filterdispersername = "BG40~holo4_003"
#filterdispersername = "FELH0600~holo4_003"

# Sources

In [14]:
datasetRefs = butler.registry.queryDatasets(datasetType='sourceTable', collections=my_collection, where= "instrument='LATISS'")

In [15]:
all_ref_for_src = []
all_dataId = []
all_src = []
all_exposures = []
all_sourcesTables = []
all_dateobs = []

for i, ref in enumerate(datasetRefs):
    all_ref_for_src.append(ref.dataId)
    print("========================datasetType = sourceTable ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit...................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_visit = ref.dataId["visit"]
    the_exposure = the_visit
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0,}
    
    # extract time
    raw_img= butler.get('raw', dataId={'exposure': the_visit, 'instrument': 'LATISS', 'detector': 0}, collections = 'LATISS/raw/all')
    meta = raw_img.getMetadata()
    md = meta.toDict()
    date_obs = md['DATE-OBS']
    
    print(the_dataId)
    df_tab=  butler.get(ref,collections=my_collection)
   
    
    
    N=len(df_tab)
    df_tab['visid_idx']= (i*np.ones(N)).astype(int)
    df_tab['dateobs']= (pd.to_datetime(date_obs))
    all_sourcesTables.append(df_tab) 
    all_dataId.append(the_dataId) 
    all_dateobs.append(date_obs)
    
    
 
    
    #registry_query = f"instrument='LATISS' and instrument=0 and visit={the_visit}]"
    #print(f"registry_query = {registry_query}")
    #info = registry.queryDimensionRecords('exposure',where = registry_query)
    
    #if i > 1:
    #    break

========================datasetType = sourceTable ============================================
fullId..................: {band: 'r', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSr_65mm~empty', visit: 2023011700190}
visit...................: 2023011700190
band....................: r
physical filter.........: SDSSr_65mm~empty
run.....................: u/dagoret/BPS_photimaging_202301/20230118T172011Z
{'day_obs': 20230117, 'seq_num': 190, 'detector': 0}
========================datasetType = sourceTable ============================================
fullId..................: {band: 'r', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSr_65mm~empty', visit: 2023011700204}
visit...................: 2023011700204
band....................: r
physical filter.........: SDSSr_65mm~empty
run.....................: u/dagoret/BPS_photimaging_202301/20230118T172011Z
{'day_obs': 20230117, 'seq_num': 204, 'detector': 0}
========================datasetType = sourceTable ================

In [16]:
len(all_sourcesTables)

86

In [17]:
all_sourcesTables[0]

,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,decl,calibFlux,calibFluxErr,ap03Flux,ap03FluxErr,ap03Flux_flag,ap06Flux,ap06FluxErr,ap06Flux_flag,ap09Flux,ap09FluxErr,ap09Flux_flag,ap12Flux,ap12FluxErr,ap12Flux_flag,ap17Flux,ap17FluxErr,ap17Flux_flag,ap25Flux,ap25FluxErr,ap25Flux_flag,ap35Flux,ap35FluxErr,ap35Flux_flag,ap50Flux,ap50FluxErr,ap50Flux_flag,ap70Flux,ap70FluxErr,ap70Flux_flag,sky,skyErr,psfFlux,psfFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,ixxDebiasedPSF,iyyDebiasedPSF,ixyDebiasedPSF,gaussianFlux,gaussianFluxErr,extendedness,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,blendedness_abs,blendedness_flag,blendedness_flag_noCentroid,blendedness_flag_noShape,apFlux_12_0_flag,apFlux_12_0_flag_apertureTruncated,apFlux_12_0_flag_sincCoeffsTruncated,apFlux_12_0_instFlux,apFlux_12_0_instFluxErr,apFlux_17_0_flag,apFlux_17_0_instFlux,apFlux_17_0_instFluxErr,extendedness_flag,footprintArea_value,jacobian_flag,jacobian_value,localBackground_instFlux,localBackground_instFluxErr,localBackground_flag,localBackground_flag_noGoodPixels,localBackground_flag_noPsf,pixelFlags_bad,pixelFlags_cr,pixelFlags_crCenter,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_interpolatedCenter,pixelFlags_offimage,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,psfFlux_apCorr,psfFlux_apCorrErr,psfFlux_area,psfFlux_flag,psfFlux_flag_apCorr,psfFlux_flag_edge,psfFlux_flag_noGoodPixels,gaussianFlux_flag,centroid_flag,centroid_flag_almostNoSecondDerivative,centroid_flag_badError,centroid_flag_edge,centroid_flag_noSecondDerivative,centroid_flag_notAtMaximum,centroid_flag_resetToPeak,variance_flag,variance_flag_emptyFootprint,variance_value,calib_astrometry_used,calib_detected,calib_photometry_reserved,calib_photometry_used,calib_psf_candidate,calib_psf_reserved,calib_psf_used,deblend_deblendedAsPsf,deblend_hasStrayFlux,deblend_masked,deblend_nChild,deblend_parentTooBig,deblend_patchedTemplate,deblend_rampedTemplate,deblend_skipped,deblend_tooManyPeaks,hsmPsfMoments_flag,hsmPsfMoments_flag_no_pixels,hsmPsfMoments_flag_not_contained,hsmPsfMoments_flag_parent_source,iDebiasedPSF_flag,iDebiasedPSF_flag_no_pixels,iDebiasedPSF_flag_not_contained,iDebiasedPSF_flag_parent_source,iDebiasedPSF_flag_galsim,iDebiasedPSF_flag_edge,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,visid_idx,dateobs
sourceId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2121281516538429441,89.420770,-52.691373,2023011700190,0,1132.000000,29.000000,NaN,NaN,89.420770,-52.691373,5.587857e+04,8569.382574,5.785823e+03,2110.598780,True,2.557392e+04,4304.866158,True,4.001419e+04,6436.474493,True,5.587857e+04,8569.382574,True,8.972793e+04,12159.191903,True,1.629209e+05,17969.351603,True,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,45.740842,290.756411,4.278188e+04,6534.851048,41.907017,263.555566,3.971718,24.331992,25.005683,-2.041490,18.234915,31.816237,-0.228198,1.123577e+05,13960.308326,NaN,40.897896,False,0.042507,False,5.617635e-09,4.638620e-07,-5.645283e-09,4.639253e-07,0.000000,True,True,True,True,False,True,1366.294434,209.526321,True,2193.949677,297.297303,True,1366,False,0.228938,1.118416,7.109324,True,False,False,True,False,False,True,True,True,False,False,False,False,False,0.952330,0.0,280.818604,True,False,False,False,False,True,False,False,False,False,True,False,False,False,97.983398,False,False,False,False,False,False,False,False,False,False,0,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0,2023-01-18 01:50:38.389
2121281516538429442,89.4

In [18]:
df=pd.concat(all_sourcesTables)

In [19]:
df

,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,decl,calibFlux,calibFluxErr,ap03Flux,ap03FluxErr,ap03Flux_flag,ap06Flux,ap06FluxErr,ap06Flux_flag,ap09Flux,ap09FluxErr,ap09Flux_flag,ap12Flux,ap12FluxErr,ap12Flux_flag,ap17Flux,ap17FluxErr,ap17Flux_flag,ap25Flux,ap25FluxErr,ap25Flux_flag,ap35Flux,ap35FluxErr,ap35Flux_flag,ap50Flux,ap50FluxErr,ap50Flux_flag,ap70Flux,ap70FluxErr,ap70Flux_flag,sky,skyErr,psfFlux,psfFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,ixxDebiasedPSF,iyyDebiasedPSF,ixyDebiasedPSF,gaussianFlux,gaussianFluxErr,extendedness,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,blendedness_abs,blendedness_flag,blendedness_flag_noCentroid,blendedness_flag_noShape,apFlux_12_0_flag,apFlux_12_0_flag_apertureTruncated,apFlux_12_0_flag_sincCoeffsTruncated,apFlux_12_0_instFlux,apFlux_12_0_instFluxErr,apFlux_17_0_flag,apFlux_17_0_instFlux,apFlux_17_0_instFluxErr,extendedness_flag,footprintArea_value,jacobian_flag,jacobian_value,localBackground_instFlux,localBackground_instFluxErr,localBackground_flag,localBackground_flag_noGoodPixels,localBackground_flag_noPsf,pixelFlags_bad,pixelFlags_cr,pixelFlags_crCenter,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_interpolatedCenter,pixelFlags_offimage,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,psfFlux_apCorr,psfFlux_apCorrErr,psfFlux_area,psfFlux_flag,psfFlux_flag_apCorr,psfFlux_flag_edge,psfFlux_flag_noGoodPixels,gaussianFlux_flag,centroid_flag,centroid_flag_almostNoSecondDerivative,centroid_flag_badError,centroid_flag_edge,centroid_flag_noSecondDerivative,centroid_flag_notAtMaximum,centroid_flag_resetToPeak,variance_flag,variance_flag_emptyFootprint,variance_value,calib_astrometry_used,calib_detected,calib_photometry_reserved,calib_photometry_used,calib_psf_candidate,calib_psf_reserved,calib_psf_used,deblend_deblendedAsPsf,deblend_hasStrayFlux,deblend_masked,deblend_nChild,deblend_parentTooBig,deblend_patchedTemplate,deblend_rampedTemplate,deblend_skipped,deblend_tooManyPeaks,hsmPsfMoments_flag,hsmPsfMoments_flag_no_pixels,hsmPsfMoments_flag_not_contained,hsmPsfMoments_flag_parent_source,iDebiasedPSF_flag,iDebiasedPSF_flag_no_pixels,iDebiasedPSF_flag_not_contained,iDebiasedPSF_flag_parent_source,iDebiasedPSF_flag_galsim,iDebiasedPSF_flag_edge,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,visid_idx,dateobs
sourceId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2121281516538429441,89.420770,-52.691373,2023011700190,0,1132.000000,29.000000,NaN,NaN,89.420770,-52.691373,5.587857e+04,8569.382574,5.785823e+03,2110.598780,True,2.557392e+04,4304.866158,True,4.001419e+04,6436.474493,True,5.587857e+04,8569.382574,True,8.972793e+04,12159.191903,True,1.629209e+05,17969.351603,True,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,45.740842,290.756411,4.278188e+04,6534.851048,41.907017,263.555566,3.971718,24.331992,25.005683,-2.041490,18.234915,31.816237,-0.228198,1.123577e+05,13960.308326,NaN,40.897896,False,0.042507,False,5.617635e-09,4.638620e-07,-5.645283e-09,4.639253e-07,0.000000,True,True,True,True,False,True,1366.294434,209.526321,True,2193.949677,297.297303,True,1366,False,0.228938,1.118416,7.109324,True,False,False,True,False,False,True,True,True,False,False,False,False,False,0.952330,0.0,280.818604,True,False,False,False,False,True,False,False,False,False,True,False,False,False,97.983398,False,False,False,False,False,False,False,False,False,False,0,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,2023011700190,0,r,SDSSr_65mm~empty,0,2023-01-18 01:50:38.389
2121281516538429442,89.4

In [20]:
df.to_csv(file_out)

In [21]:
print(df.columns.tolist())

['coord_ra', 'coord_dec', 'ccdVisitId', 'parentSourceId', 'x', 'y', 'xErr', 'yErr', 'ra', 'decl', 'calibFlux', 'calibFluxErr', 'ap03Flux', 'ap03FluxErr', 'ap03Flux_flag', 'ap06Flux', 'ap06FluxErr', 'ap06Flux_flag', 'ap09Flux', 'ap09FluxErr', 'ap09Flux_flag', 'ap12Flux', 'ap12FluxErr', 'ap12Flux_flag', 'ap17Flux', 'ap17FluxErr', 'ap17Flux_flag', 'ap25Flux', 'ap25FluxErr', 'ap25Flux_flag', 'ap35Flux', 'ap35FluxErr', 'ap35Flux_flag', 'ap50Flux', 'ap50FluxErr', 'ap50Flux_flag', 'ap70Flux', 'ap70FluxErr', 'ap70Flux_flag', 'sky', 'skyErr', 'psfFlux', 'psfFluxErr', 'ixx', 'iyy', 'ixy', 'ixxPSF', 'iyyPSF', 'ixyPSF', 'ixxDebiasedPSF', 'iyyDebiasedPSF', 'ixyDebiasedPSF', 'gaussianFlux', 'gaussianFluxErr', 'extendedness', 'localPhotoCalib', 'localPhotoCalib_flag', 'localPhotoCalibErr', 'localWcs_flag', 'localWcs_CDMatrix_2_1', 'localWcs_CDMatrix_1_1', 'localWcs_CDMatrix_1_2', 'localWcs_CDMatrix_2_2', 'blendedness_abs', 'blendedness_flag', 'blendedness_flag_noCentroid', 'blendedness_flag_noShape',

In [22]:
df['dateobs'].unique()

array(['2023-01-18T01:50:38.389000000', '2023-01-18T02:03:10.916000000',
       '2023-01-18T00:52:53.386000000', '2023-01-18T01:09:05.412000000',
       '2023-01-18T02:21:06.342000000', '2023-01-18T01:06:54.786000000',
       '2023-01-18T01:24:38.942000000', '2023-01-18T02:20:33.730000000',
       '2023-01-18T02:06:25.906000000', '2023-01-18T01:08:14.286000000',
       '2023-01-18T02:16:32.816000000', '2023-01-18T01:04:06.192000000',
       '2023-01-18T02:11:45.334000000', '2023-01-18T01:13:16.362000000',
       '2023-01-18T02:09:53.469000000', '2023-01-18T02:21:54.720000000',
       '2023-01-18T01:11:01.184000000', '2023-01-18T00:52:20.759000000',
       '2023-01-18T02:01:50.518000000', '2023-01-18T01:50:05.764000000',
       '2023-01-18T01:22:47.312000000', '2023-01-18T01:10:28.559000000',
       '2023-01-18T01:06:22.169000000', '2023-01-18T01:25:27.559000000',
       '2023-01-18T02:12:33.205000000', '2023-01-18T00:59:21.257000000',
       '2023-01-18T02:00:30.070000000', '2023-01-18